# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_info = pd.read_csv("cities.csv")
weather_info.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Colonia Carlos Antonio Lopez,0,PY,1580073397,24,-26.40,-54.77,94.1,3.13
1,Puerto Ayora,75,EC,1580073174,83,-0.74,-90.35,78.8,2.24
2,Saint-Philippe,90,RE,1580073196,73,-21.36,55.77,77.0,10.29
3,Longyearbyen,0,SJ,1580073156,77,78.22,15.64,1.4,1.12
4,Ushuaia,75,AR,1580072911,76,-54.80,-68.30,50.0,27.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_info[["Lat", "Lng"]]
humidity = weather_info["Humidity"]

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
narrowed_city_df = weather_info.loc[(weather_info["Wind Speed"] <= 10) & (weather_info["Cloudiness"] == 0) & \
                                   (weather_info["Max Temp"] >= 70) & (weather_info["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
143,Taoudenni,0,ML,1580073415,22,22.68,-3.98,75.69,9.98
192,Castro,0,BR,1580073217,48,-24.79,-50.01,79.25,0.81
215,Mangrol,0,IN,1580073425,62,21.12,70.12,72.10,9.31
244,Leo,0,BF,1580073428,8,11.10,-2.10,76.66,8.61
286,Mossamedes,0,AO,1580073434,77,-15.20,12.15,75.06,8.30
369,Say,0,NE,1580073445,9,13.10,2.37,77.00,8.05
391,Caborca,0,MX,1580073305,33,30.58,-111.10,75.99,4.00
429,Yulara,0,AU,1580073452,24,-25.24,130.99,71.60,5.82
458,Bumba,0,CD,1580073455,38,2.18,22.47,72.30,0.76
556,Tonj,0,SS,1580073469,28,7.27,28.68,74.21,6.26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
143,Taoudenni,ML,22.68,-3.98,
192,Castro,BR,-24.79,-50.01,
215,Mangrol,IN,21.12,70.12,
244,Leo,BF,11.10,-2.10,
286,Mossamedes,AO,-15.20,12.15,
369,Say,NE,13.10,2.37,
391,Caborca,MX,30.58,-111.10,
429,Yulara,AU,-25.24,130.99,
458,Bumba,CD,2.18,22.47,
556,Tonj,SS,7.27,28.68,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{lat},{lng}"

    print("------------------")
    print("Beginning Search")
    print("------------------")    
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Skip, Not enough data.")
        
    time.sleep(1)

print("-----------------------------")
print("Search Concluded")
print("-----------------------------")

------------------
Beginning Search
------------------
Retrieving Results for Index 143: Taoudenni.
Skip, Not enough data.
------------------
Beginning Search
------------------
Retrieving Results for Index 192: Castro.
Closest hotel in Castro is Hotel Buganville Express.
------------------
Beginning Search
------------------
Retrieving Results for Index 215: Mangrol.
Closest hotel in Mangrol is Kaveri Hotel.
------------------
Beginning Search
------------------
Retrieving Results for Index 244: Leo.
Closest hotel in Leo is Hotel Sissilis.
------------------
Beginning Search
------------------
Retrieving Results for Index 286: Mossamedes.
Closest hotel in Mossamedes is Hotel ibis Styles Iu Namibe.
------------------
Beginning Search
------------------
Retrieving Results for Index 369: Say.
Skip, Not enough data.
------------------
Beginning Search
------------------
Retrieving Results for Index 391: Caborca.
Closest hotel in Caborca is Motel Elba.
------------------
Beginning Search
-

In [7]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
143,Taoudenni,ML,22.68,-3.98,
192,Castro,BR,-24.79,-50.01,Hotel Buganville Express
215,Mangrol,IN,21.12,70.12,Kaveri Hotel
244,Leo,BF,11.10,-2.10,Hotel Sissilis
286,Mossamedes,AO,-15.20,12.15,Hotel ibis Styles Iu Namibe
369,Say,NE,13.10,2.37,
391,Caborca,MX,30.58,-111.10,Motel Elba
429,Yulara,AU,-25.24,130.99,Sails in the Desert
458,Bumba,CD,2.18,22.47,
556,Tonj,SS,7.27,28.68,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))